# Logistic Route Optimization - Illinois Idition

## Project Scope
This notebook performs intercity delivery route optimization across key cities in the state of Illinois, with a central logistics hub in Chicago. It uses OpenStreetMap road network data to build a realistic driving graph and applies advanced algorithms to compute optimal delivery routes.

### Goals
- Map delivery cities to the road network<br>
- Calculate travel distances between cities<br>
- Solve for the most efficient route using Google's OR-Tools<br>
- Visualize results using interactive maps<br>

### Load the Dataset and Display Cities

Load Illinois cities dataset from CSV. If not found, generate it using the script.

In [1]:
import os
import sys

# Add 'src' to sys.path if it's not already there
src_path = os.path.abspath('../src/')
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
import pandas as pd

csv_path = '../data/illinois_cities.csv'

try:
    df_cities = pd.read_csv(csv_path)
except FileNotFoundError:
    # Import the function
    from cities_dataset_generator import generate_illinois_cities_dataset

    # Generate the dataset
    generate_illinois_cities_dataset(csv_path)

    # Read it after creation
    df_cities = pd.read_csv(csv_path)

df_cities

,city,lat,lon
0,Chicago,41.8781,-87.6298
1,Aurora,41.7606,-88.3201
2,Rockford,42.2711,-89.0937
3,Naperville,41.7508,-88.1535
4,Joliet,41.5250,-88.0817
5,Springfield,39.7817,-89.6501
6,Peoria,40.6936,-89.5889
7,Champaign,40.1164,-88.2434


Generate an intercative map of Illinois with city markers using `folium`

In [3]:
import folium

# Get center of Illinois
map_center = [df_cities['lat'].mean(), df_cities['lon'].mean()]

# Create base map
m = folium.Map(location=map_center, zoom_start=7)

# Add each city as a marker
for _, row in df_cities.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['city'],
        icon=folium.Icon(color='blue', icon='info-sign'),
    ).add_to(m)

m

### Generate and Save Road Network

Download Illinois driving network with `osmnx`. If not found, generate it using the script.

In [4]:
import osmnx as ox

graph_path = '../data/illinois_graph.graphml'

try:
    G = ox.load_graphml(graph_path)
except FileNotFoundError:
    # Import the function
    from graphml_generator import generate_graphml

    # Genrate the graph
    generate_graphml(df_path='../data/illinois_cities.csv', file_path=graph_path)

    # Load after creation
    G = ox.load_graphml(graph_path)

### Match Cities to Nearest Road Nodes

In [5]:
# Extract lat/lon from the dataframe
city_coords = list(zip(df_cities['lat'], df_cities['lon']))

# Get the nearest graph node for each city
city_node_ids = []
for lat, lon in city_coords:
    node = ox.distance.nearest_nodes(G, X=lon, Y=lat)
    city_node_ids.append(node)

df_cities['node_id'] = city_node_ids
df_cities

,city,lat,lon,node_id
0,Chicago,41.8781,-87.6298,261154311
1,Aurora,41.7606,-88.3201,235209482
2,Rockford,42.2711,-89.0937,237540096
3,Naperville,41.7508,-88.1535,237638509
4,Joliet,41.5250,-88.0817,237451309
5,Springfield,39.7817,-89.6501,1374889316
6,Peoria,40.6936,-89.5889,236827183
7,Champaign,40.1164,-88.2434,38035941


### Compute the Road Distance Matrix Between Cities in Miles

By default, `OSMnx` and `NetworkX` use meters for edge lengths (`length` attribute), as OpenStreetMap stores distances in meters globally.
Since this project focuses on the United States, it's more appropriate to convert distances to miles, which is the standard unit in U.S. logistics.

In [6]:
import networkx as nx

# Prepare distance matrix
cities = df_cities['city'].tolist()
node_ids = df_cities['node_id'].tolist()

# Initialize empty DataFrame
distance_matrix = pd.DataFrame(index=cities, columns=cities, dtype=float)

# Compute distances
for i, (city_a, node_a) in enumerate(zip(cities, node_ids)):
    for j, (city_b, node_b) in enumerate(zip(cities, node_ids)):
        if city_a == city_b:
            distance_matrix.loc[city_a, city_b] = 0.0
        else:
            try:
                length = nx.shortest_path_length(G, node_a, node_b, weight='length')
                distance_matrix.loc[city_a, city_b] = round(length / 1609.34, 2)  # convert to miles
            except nx.NetworkXNoPath:
                distance_matrix.loc[city_a, city_b] = float('inf')  # No route

distance_matrix

,Chicago,Aurora,Rockford,Naperville,Joliet,Springfield,Peoria,Champaign
Chicago,0.00,38.09,83.26,30.22,37.30,191.48,147.64,131.12
Aurora,38.15,0.00,60.75,9.84,22.38,166.01,112.41,120.51
Rockford,83.57,60.77,0.00,67.61,83.06,190.50,123.78,176.32
Naperville,30.27,9.76,67.55,0.00,18.30,168.21,118.05,117.81
Joliet,37.34,22.46,82.88,18.30,0.00,154.41,112.95,103.76
Springfield,191.20,165.71,190.77,168.06,154.02,0.00,69.01,84.95
Peoria,147.86,112.59,123.88,118.48,113.04,68.86,0.00,87.83
Champaign,131.60,120.44,176.24,117.96,103.47,85.12,87.75,0.00


Save the matrix

In [7]:
distance_matrix.to_csv('../data/distance_matrix.csv')

### Optimize Delivery Route (TSP with OR-Tools)

We use Google's OR-Tools to compute the shortest route that visits all cities exactly once and returns to the start.

In [8]:
import json

result_path = '../data/tsp_result.json'

try:
    with open(result_path, 'r') as f:
        tsp_result = json.load(f)
except FileNotFoundError:
    %run ../src/tsp_solver.py
    with open(result_path, 'r') as f:
        tsp_result = json.load(f)

print('Optimal route:')
print(' -> '.join(tsp_result['route']))
print(f"Total distance: {tsp_result['total_distance']} miles")

Optimal route:
Chicago -> Naperville -> Aurora -> Rockford -> Peoria -> Springfield -> Champaign -> Joliet -> Chicago
Total distance: 519.13 miles
